In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from uszipcode import SearchEngine
from IPython.display import Image
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Linear Regression w/ l2 norm (Ridge)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

# Import Data
Get data from different sources before combining
* Cleaned up EV data: TX_WA_CO_NY.csv
* Average EV price and new car data over time: Avg_EV_Price.csv
* Census data (pop, household income, zipcode): census.csv 

In [60]:
# Import data
df_reg = pd.read_csv('./Data/TX_WA_CO_NY.csv')
df_ev = pd.read_csv('./Data/Avg_EV_Price.csv')
df_c = pd.read_csv('./Data/Census Data/census.csv')

# Convert dates to datetime dtype
df_reg['Registration Date'] = pd.to_datetime(df_reg['Registration Date'])
df_ev['Month'] = pd.to_datetime(df_ev['Month'], format='%b-%y')


# Merge Data
## Aggregate Registration Data by County

In [61]:
#Aggregate by County
# create a SearchEngine object
search = SearchEngine()

# define a function to map zip codes to counties
def zipcode_to_county(zipcode):
    #This county does not get populated for some reason
    if zipcode == 75033:
        return "Collin County"
    
    zipcode_data = search.by_zipcode(zipcode)
    county = zipcode_data.county
    return county

# apply the function to create a new column "County"
df_reg['County'] = df_reg['ZIP Code'].apply(zipcode_to_county)

In [62]:
df_reg.shape

(264426, 6)

In [63]:
nan_rows = df_reg[df_reg.isna().any(axis=1)]

In [64]:
df_reg = df_reg.groupby(["State", "Registration Date", "Drivetrain Type", "County"]).agg('sum').drop(columns = ["ZIP Code"]).reset_index()

start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2021-12-31')
df_reg = df_reg[(df_reg['Registration Date'] >= start_date) & (df_reg['Registration Date'] <= end_date)]

In [65]:
df_reg[df_reg['County'] == ""]
df_reg.shape

(19398, 5)

## EV and New Car Prices

In [66]:
# merge ev data in main df
df_reg_ev = pd.merge(df_reg, df_ev, left_on='Registration Date', right_on='Month', how='left')
df_reg_ev = df_reg_ev.drop(['Month'], axis=1)

# Since we don't have ev price data for earlier dates, set all NaN to price for 2020-01-01
fill_val = {'Average EV Price' : df_ev['Average EV Price'][0], 'New Car Average' : df_ev['New Car Average'][0]}
df_reg_ev = df_reg_ev.fillna(value=fill_val)

In [67]:
# check if any nan values
nan_rows = df_reg_ev[df_reg_ev.isna().any(axis=1)]
nan_rows
df_reg_ev.shape

(19398, 7)

## Census Data

In [68]:
# merge census data
df_reg_ev_c = pd.merge(df_reg_ev, df_c, left_on=['County', "State"], right_on=['county', 'state'], how='left')

df_reg_ev_c = df_reg_ev_c.drop(['Unnamed: 0', 'county'], axis=1)

# check if any nan values
nan_rows = df_reg_ev_c[df_reg_ev_c.isna().any(axis=1)]

print(nan_rows)

# Extract info of missing census data
print('Missing census data in:')
print('counties = ',nan_rows['County'].unique())
print('states = ',nan_rows['State'].unique())
print('Total num of countires = ',len(nan_rows['County'].unique()))
print('Total entries w/ nan = ', len(nan_rows))
#print('Total entries in df = ', len(df))

# The number of missing data is < 1% of total data, just drop
df_reg_ev_c = df_reg_ev_c.dropna()

df_reg_ev_c[['population', 'household_income']] = df_reg_ev_c[['population', 'household_income']].astype(int)


Empty DataFrame
Columns: [State, Registration Date, Drivetrain Type, County, Vehicle Count, Average EV Price, New Car Average, Unnamed: 0.1, population, household_income, state]
Index: []
Missing census data in:
counties =  []
states =  []
Total num of countires =  0
Total entries w/ nan =  0


In [69]:
print(df_reg_ev_c.dtypes)

State                        object
Registration Date    datetime64[ns]
Drivetrain Type              object
County                       object
Vehicle Count                 int64
Average EV Price             object
New Car Average              object
Unnamed: 0.1                  int64
population                    int32
household_income              int32
state                        object
dtype: object


# Urban/Rural Divide
Source: https://www2.census.gov/geo/docs/reference/ua/2020_UA_COUNTY.xlsx 
Website: https://www.census.gov/programs-surveys/geography/guidance/geo-areas/urban-rural.html

In [70]:
county_pop_density = pd.read_excel("./Data/2020_UA_COUNTY.xlsx")#, sheet = "2020_UA_COUNTY")
county_pop_density['STATE_NAME']
def state_to_abbreviation(state):
    if state == 'Texas':
        return 'TX'
    elif state == 'New York':
        return 'NY'
    elif state == 'Colorado':
        return 'CO'
    elif state == 'Washington':
        return 'WA'
    else:
        return None # or whatever you want to return if the input is not a valid state name

county_pop_density["STATE_NAME"] = county_pop_density["STATE_NAME"].apply(state_to_abbreviation)

In [71]:
county_pop_density_df = county_pop_density[["STATE_NAME","COUNTY_NAME", "POPDEN_COU"]]
county_pop_density_df['urban_flag'] = [1 if x > 500 else 0 for x in county_pop_density_df['POPDEN_COU']]
county_pop_density_df['COUNTY_NAME'] = county_pop_density_df['COUNTY_NAME'].apply(lambda x: x + ' County')
county_pop_density_df

C:\Users\markm\AppData\Local\Temp\ipykernel_19496\3497240639.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_pop_density_df['urban_flag'] = [1 if x > 500 else 0 for x in county_pop_density_df['POPDEN_COU']]
C:\Users\markm\AppData\Local\Temp\ipykernel_19496\3497240639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_pop_density_df['COUNTY_NAME'] = county_pop_density_df['COUNTY_NAME'].apply(lambda x: x + ' County')


,STATE_NAME,COUNTY_NAME,POPDEN_COU,urban_flag
0,None,Autauga County,98.922916,0
1,None,Baldwin County,145.781265,0
2,None,Barbour County,28.500467,0
3,None,Bibb County,35.814001,0
4,None,Blount County,91.696680,0
...,...,...,...,...
3229,None,Yabucoa County,551.052867,1
3230,None,Yauco County,504.673516,1
3231,None,St. Croix County,491.862425,0
3232,None,St. John County,197.101643,0


In [72]:
df_gas_elec_prices = pd.read_csv("./Data/electricity_gas_prices_reformatted.csv")

# Define a dictionary to map month names to numerical values
month_dict = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 
              'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 
              'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
urban_rural_dict = {"Urban" : 1.0, "Rural" : 0.0}

# Use the map() method to convert the "month" column to numerical values
df_gas_elec_prices['Month'] = df_gas_elec_prices['Month'].map(month_dict)

df_gas_elec_prices['urban_flag'] = df_gas_elec_prices['Population Type'].map(urban_rural_dict)


# Combine "year" and "month" columns into a new column in the format "YYYY-MM"
df_gas_elec_prices['year_month'] = df_gas_elec_prices['Year'].astype(str) + '-' + df_gas_elec_prices['Month']

# Convert the "year_month" column to a datetime object
df_gas_elec_prices['date'] = pd.to_datetime(df_gas_elec_prices['year_month'])


df_gas_elec_prices = df_gas_elec_prices.drop(columns = ["Unnamed: 0", "Population Type", "Year", 
                                            "Month", "year_month"])

df_gas_elec_prices


,State,Electricity Price,Gas Price,urban_flag,date
0,TX,10.490,2.139,0.0,2017-01-01
1,TX,10.490,2.083,0.0,2017-02-01
2,TX,10.490,2.089,0.0,2017-03-01
3,TX,10.490,2.195,0.0,2017-04-01
4,TX,10.490,2.187,0.0,2017-05-01
...,...,...,...,...,...
531,CO,0.157,3.691,1.0,2022-10-01
532,CO,0.152,3.429,1.0,2022-11-01
533,CO,0.152,2.979,1.0,2022-12-01
534,CO,0.153,3.479,1.0,2023-01-01


In [73]:
df = pd.merge(df_reg_ev_c, county_pop_density_df, left_on=['State','County'], right_on=['STATE_NAME','COUNTY_NAME'], how='left')
df = pd.merge(df, df_gas_elec_prices, left_on = ["State", "urban_flag" ,"Registration Date"], right_on = ["State", "urban_flag", "date"], how = "left")
df = df.drop(columns = ["Unnamed: 0.1", "STATE_NAME", "COUNTY_NAME", "date", "urban_flag"])
nan_rows = df[df.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [State, Registration Date, Drivetrain Type, County, Vehicle Count, Average EV Price, New Car Average, population, household_income, state, POPDEN_COU, Electricity Price, Gas Price]
Index: []


In [74]:
df

,State,Registration Date,Drivetrain Type,County,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price
0,CO,2017-01-01,BEV,Adams County,24,"$54,669","$38,747",509844,167290,CO,445.323695,12.66,3.378
1,CO,2017-01-01,BEV,Arapahoe County,65,"$54,669","$38,747",649980,241889,CO,821.038725,0.12,2.429
2,CO,2017-01-01,BEV,Boulder County,167,"$54,669","$38,747",324682,127365,CO,455.351666,12.66,3.378
3,CO,2017-01-01,BEV,Broomfield County,14,"$54,669","$38,747",69444,27199,CO,2248.011733,0.12,2.429
4,CO,2017-01-01,BEV,Denver County,94,"$54,669","$38,747",715878,287756,CO,4674.337363,0.12,2.429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,WA,2021-12-01,PHEV,Wahkiakum County,6,"$63,821","$47,243",4318,1900,WA,16.818489,9.72,3.908
19394,WA,2021-12-01,PHEV,Walla Walla County,81,"$63,821","$47,243",60785,22773,WA,49.277735,9.72,3.908
19395,WA,2021-12-01,PHEV,Whatcom County,639,"$63,821","$47,243",224538,88978,WA,107.617198,9.72,3.908
19396,WA,2021-12-01,PHEV,Whitman County,56,"$63,821","$47,243",49577,18485,WA,22.217436,9.72,3.908


# Export Data

In [75]:
# Desired prediction var
predict_label = 'Vehicle Count'

# Drop zip code since it would increase the number of features by ~4k
# also zip code is highly correlated to population and income
# Drop registration date since we will change to Unix timestamps
drop_col = ['state','County', 'Registration Date']

# Assemble Categorical Variables
cat_var = ['State', 'Drivetrain Type']
for cat in cat_var:
    # Get dummy variables for cat
    dummy_var = df[cat].unique()
    dummy_var = dummy_var[1:]

    # create df w/ one hot cat features
    df_cat = pd.get_dummies(df[cat], drop_first=True)

    # drop original
    df = df.drop([cat], axis=1)

    # concatenate
    df = pd.concat([df, df_cat], axis=1)

# Add Unix timestamp   
df['Unix Time'] = df['Registration Date'].apply(lambda x: x.timestamp())

# Convert $ price into integer
df['Average EV Price'] = df['Average EV Price'].str.replace('$','').str.replace(',','').astype(int)
df['New Car Average'] = df['New Car Average'].str.replace('$','').str.replace(',','').astype(int)

df

C:\Users\markm\AppData\Local\Temp\ipykernel_19496\3669364176.py:29: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Average EV Price'] = df['Average EV Price'].str.replace('$','').str.replace(',','').astype(int)
C:\Users\markm\AppData\Local\Temp\ipykernel_19496\3669364176.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['New Car Average'] = df['New Car Average'].str.replace('$','').str.replace(',','').astype(int)


,Registration Date,County,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,PHEV,Unix Time
0,2017-01-01,Adams County,24,54669,38747,509844,167290,CO,445.323695,12.66,3.378,0,0,0,0,1.483229e+09
1,2017-01-01,Arapahoe County,65,54669,38747,649980,241889,CO,821.038725,0.12,2.429,0,0,0,0,1.483229e+09
2,2017-01-01,Boulder County,167,54669,38747,324682,127365,CO,455.351666,12.66,3.378,0,0,0,0,1.483229e+09
3,2017-01-01,Broomfield County,14,54669,38747,69444,27199,CO,2248.011733,0.12,2.429,0,0,0,0,1.483229e+09
4,2017-01-01,Denver County,94,54669,38747,715878,287756,CO,4674.337363,0.12,2.429,0,0,0,0,1.483229e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2021-12-01,Wahkiakum County,6,63821,47243,4318,1900,WA,16.818489,9.72,3.908,0,0,1,1,1.638317e+09
19394,2021-12-01,Walla Walla County,81,63821,47243,60785,22773,WA,49.277735,9.72,3.908,0,0,1,1,1.638317e+09
19395,2021-12-01,Whatcom County,639,63821,47243,224538,88978,WA,107.617198,9.72,3.908,0,0,1,1,1.638317e+09
19396,2021-12-01,Whitman County,56,63821,47243,49577,18485,WA,22.217436,9.72,3.908,0,0,1,1,1.638317e+09


In [82]:
# Desired prediction var
predict_label = 'Vehicle Count'

# Get labels of all features
features = [c for c in df.columns if c not in [predict_label] + drop_col]
features
# extract values to np
y = df[predict_label].to_numpy()
X = df[features].to_numpy()

df = df.reset_index(drop=True)
df.to_csv('./Data/df_all_features_county.csv', index = False)
df[features].to_csv('./Data/df_X_county.csv', index = False)
df[predict_label].to_csv('./Data/df_y_county.csv', index = False)


# Add Fips and Predict both PHEV and BEV
Currently we have Vehicle Count we are predicting with a one hot vector for PHEV

However, we actually want to predict both of these values so we should have two columns per them

Essentially we have duplicate nodes that we need to get rid of for the GNN

In [114]:
df = pd.read_csv('./Data/df_all_features_county.csv')
df

,Registration Date,County,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,PHEV,Unix Time,fips
0,2017-01-01,Adams County,24,54669,38747,509844,167290,CO,445.323695,12.66,3.378,0,0,0,0,1.483229e+09,8001
1,2017-01-01,Arapahoe County,65,54669,38747,649980,241889,CO,821.038725,0.12,2.429,0,0,0,0,1.483229e+09,8005
2,2017-01-01,Boulder County,167,54669,38747,324682,127365,CO,455.351666,12.66,3.378,0,0,0,0,1.483229e+09,8013
3,2017-01-01,Broomfield County,14,54669,38747,69444,27199,CO,2248.011733,0.12,2.429,0,0,0,0,1.483229e+09,8014
4,2017-01-01,Denver County,94,54669,38747,715878,287756,CO,4674.337363,0.12,2.429,0,0,0,0,1.483229e+09,8031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2021-12-01,Wahkiakum County,6,63821,47243,4318,1900,WA,16.818489,9.72,3.908,0,0,1,1,1.638317e+09,53069
19394,2021-12-01,Walla Walla County,81,63821,47243,60785,22773,WA,49.277735,9.72,3.908,0,0,1,1,1.638317e+09,53071
19395,2021-12-01,Whatcom County,639,63821,47243,224538,88978,WA,107.617198,9.72,3.908,0,0,1,1,1.638317e+09,53073
19396,2021-12-01,Whitman County,56,63821,47243,49577,18485,WA,22.217436,9.72,3.908,0,0,1,1,1.638317e+09,53075


In [115]:
state_list = df['state'].to_list()
county_list = df['County'].to_list()
county_state_list = [county_list[i] + ', ' + state_list[i] for i in range(len(state_list))]
county_state_list

['Adams County, CO',
 'Arapahoe County, CO',
 'Boulder County, CO',
 'Broomfield County, CO',
 'Denver County, CO',
 'Douglas County, CO',
 'Eagle County, CO',
 'El Paso County, CO',
 'Elbert County, CO',
 'Garfield County, CO',
 'Grand County, CO',
 'Gunnison County, CO',
 'Jefferson County, CO',
 'La Plata County, CO',
 'Larimer County, CO',
 'Las Animas County, CO',
 'Mesa County, CO',
 'Montrose County, CO',
 'Otero County, CO',
 'Park County, CO',
 'Pitkin County, CO',
 'Pueblo County, CO',
 'San Miguel County, CO',
 'Summit County, CO',
 'Weld County, CO',
 'Adams County, CO',
 'Alamosa County, CO',
 'Arapahoe County, CO',
 'Boulder County, CO',
 'Broomfield County, CO',
 'Delta County, CO',
 'Denver County, CO',
 'Douglas County, CO',
 'Eagle County, CO',
 'El Paso County, CO',
 'Elbert County, CO',
 'Garfield County, CO',
 'Gunnison County, CO',
 'Hinsdale County, CO',
 'Jefferson County, CO',
 'La Plata County, CO',
 'Lake County, CO',
 'Larimer County, CO',
 'Logan County, CO

In [91]:
# Create a dictionary that maps the 'fruit' column of the DataFrame to the 'color' column
map_dict = dict(zip(df_a['countyname'], df_a['fipscounty']))

# Use the map() function to map the values in the list to the corresponding values in the DataFrame column
mapped_fips = list(map(lambda f: map_dict[f], county_state_list))
df['fips'] = mapped_fips
df.to_csv('./Data/df_all_features_county.csv', index = False)
df

,Registration Date,County,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,PHEV,Unix Time,fips
0,2017-01-01,Adams County,24,54669,38747,509844,167290,CO,445.323695,12.66,3.378,0,0,0,0,1.483229e+09,8001
1,2017-01-01,Arapahoe County,65,54669,38747,649980,241889,CO,821.038725,0.12,2.429,0,0,0,0,1.483229e+09,8005
2,2017-01-01,Boulder County,167,54669,38747,324682,127365,CO,455.351666,12.66,3.378,0,0,0,0,1.483229e+09,8013
3,2017-01-01,Broomfield County,14,54669,38747,69444,27199,CO,2248.011733,0.12,2.429,0,0,0,0,1.483229e+09,8014
4,2017-01-01,Denver County,94,54669,38747,715878,287756,CO,4674.337363,0.12,2.429,0,0,0,0,1.483229e+09,8031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2021-12-01,Wahkiakum County,6,63821,47243,4318,1900,WA,16.818489,9.72,3.908,0,0,1,1,1.638317e+09,53069
19394,2021-12-01,Walla Walla County,81,63821,47243,60785,22773,WA,49.277735,9.72,3.908,0,0,1,1,1.638317e+09,53071
19395,2021-12-01,Whatcom County,639,63821,47243,224538,88978,WA,107.617198,9.72,3.908,0,0,1,1,1.638317e+09,53073
19396,2021-12-01,Whitman County,56,63821,47243,49577,18485,WA,22.217436,9.72,3.908,0,0,1,1,1.638317e+09,53075


In [117]:
df_phev = df[df['PHEV'] == 1]
df_phev

,Registration Date,County,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,PHEV,Unix Time,fips
25,2017-01-01,Adams County,22,54669,38747,509844,167290,CO,445.323695,12.66,3.378,0,0,0,1,1.483229e+09,8001
26,2017-01-01,Alamosa County,1,54669,38747,16153,6240,CO,22.661310,12.66,3.378,0,0,0,1,1.483229e+09,8003
27,2017-01-01,Arapahoe County,61,54669,38747,649980,241889,CO,821.038725,0.12,2.429,0,0,0,1,1.483229e+09,8005
28,2017-01-01,Boulder County,63,54669,38747,324682,127365,CO,455.351666,12.66,3.378,0,0,0,1,1.483229e+09,8013
29,2017-01-01,Broomfield County,7,54669,38747,69444,27199,CO,2248.011733,0.12,2.429,0,0,0,1,1.483229e+09,8014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19393,2021-12-01,Wahkiakum County,6,63821,47243,4318,1900,WA,16.818489,9.72,3.908,0,0,1,1,1.638317e+09,53069
19394,2021-12-01,Walla Walla County,81,63821,47243,60785,22773,WA,49.277735,9.72,3.908,0,0,1,1,1.638317e+09,53071
19395,2021-12-01,Whatcom County,639,63821,47243,224538,88978,WA,107.617198,9.72,3.908,0,0,1,1,1.638317e+09,53073
19396,2021-12-01,Whitman County,56,63821,47243,49577,18485,WA,22.217436,9.72,3.908,0,0,1,1,1.638317e+09,53075


In [126]:
df_bev = df[df['PHEV']==0]
#df_bev = df_bev[['Registration Date','fips','Vehicle Count']]
df_bev

,Registration Date,County,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,PHEV,Unix Time,fips
0,2017-01-01,Adams County,24,54669,38747,509844,167290,CO,445.323695,12.66,3.378,0,0,0,0,1.483229e+09,8001
1,2017-01-01,Arapahoe County,65,54669,38747,649980,241889,CO,821.038725,0.12,2.429,0,0,0,0,1.483229e+09,8005
2,2017-01-01,Boulder County,167,54669,38747,324682,127365,CO,455.351666,12.66,3.378,0,0,0,0,1.483229e+09,8013
3,2017-01-01,Broomfield County,14,54669,38747,69444,27199,CO,2248.011733,0.12,2.429,0,0,0,0,1.483229e+09,8014
4,2017-01-01,Denver County,94,54669,38747,715878,287756,CO,4674.337363,0.12,2.429,0,0,0,0,1.483229e+09,8031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19354,2021-12-01,Wahkiakum County,15,63821,47243,4318,1900,WA,16.818489,9.72,3.908,0,0,1,0,1.638317e+09,53069
19355,2021-12-01,Walla Walla County,139,63821,47243,60785,22773,WA,49.277735,9.72,3.908,0,0,1,0,1.638317e+09,53071
19356,2021-12-01,Whatcom County,1587,63821,47243,224538,88978,WA,107.617198,9.72,3.908,0,0,1,0,1.638317e+09,53073
19357,2021-12-01,Whitman County,69,63821,47243,49577,18485,WA,22.217436,9.72,3.908,0,0,1,0,1.638317e+09,53075


In [149]:
merged_df = pd.merge(df_phev, df_bev, on=['Registration Date','fips'], how='outer')
merged_df 




,Registration Date,County_x,Vehicle Count_x,Average EV Price_x,New Car Average_x,population_x,household_income_x,state_x,POPDEN_COU_x,Electricity Price_x,...,household_income_y,state_y,POPDEN_COU_y,Electricity Price_y,Gas Price_y,NY_y,TX_y,WA_y,PHEV_y,Unix Time_y
0,2017-01-01,Adams County,22.0,54669.0,38747.0,509844.0,167290.0,CO,445.323695,12.66,...,167290.0,CO,445.323695,12.66,3.378,0.0,0.0,0.0,0.0,1.483229e+09
1,2017-01-01,Alamosa County,1.0,54669.0,38747.0,16153.0,6240.0,CO,22.661310,12.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,Arapahoe County,61.0,54669.0,38747.0,649980.0,241889.0,CO,821.038725,0.12,...,241889.0,CO,821.038725,0.12,2.429,0.0,0.0,0.0,0.0,1.483229e+09
3,2017-01-01,Boulder County,63.0,54669.0,38747.0,324682.0,127365.0,CO,455.351666,12.66,...,127365.0,CO,455.351666,12.66,3.378,0.0,0.0,0.0,0.0,1.483229e+09
4,2017-01-01,Broomfield County,7.0,54669.0,38747.0,69444.0,27199.0,CO,2248.011733,0.12,...,27199.0,CO,2248.011733,0.12,2.429,0.0,0.0,0.0,0.0,1.483229e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11084,2018-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5798.0,WA,9.570523,9.53,3.908,0.0,0.0,1.0,0.0,1.538352e+09
11085,2018-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4866.0,WA,7.258144,9.53,3.908,0.0,0.0,1.0,0.0,1.538352e+09
11086,2018-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9714.0,WA,25.026666,9.53,3.908,0.0,0.0,1.0,0.0,1.541030e+09
11087,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6035.0,WA,10.708173,9.52,3.908,0.0,0.0,1.0,0.0,1.546301e+09


In [150]:
column_names = df_phev.columns.tolist()

In [162]:
# Select rows from df1 that do not have a match in df2
df1_unmatched = merged_df[merged_df['state_y'].isna()]
df1_unmatched = df1_unmatched.dropna(axis=1)
col_dic = dict(zip(df1_unmatched.columns, column_names))
df1_unmatched = df1_unmatched.rename(columns=col_dic)
df1_unmatched.drop('PHEV', axis=1, inplace=True)
df1_unmatched = df1_unmatched.rename(columns={'Vehicle Count': 'PHEV'})
df1_unmatched['BEV'] = 0
df1_unmatched


,Registration Date,County,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time,fips,BEV
1,2017-01-01,Alamosa County,1.0,54669.0,38747.0,16153.0,6240.0,CO,22.661310,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8003,0
5,2017-01-01,Delta County,2.0,54669.0,38747.0,30758.0,12277.0,CO,27.314143,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8029,0
13,2017-01-01,Hinsdale County,1.0,54669.0,38747.0,781.0,376.0,CO,0.705319,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8053,0
16,2017-01-01,Lake County,1.0,54669.0,38747.0,7845.0,3275.0,CO,19.728484,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8065,0
18,2017-01-01,Logan County,1.0,54669.0,38747.0,22282.0,8301.0,CO,11.708985,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8075,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9717,2021-08-01,Garfield County,3.0,57540.0,43418.0,2258.0,986.0,WA,3.215953,9.72,3.908,0.0,0.0,1.0,1.627776e+09,53023,0
9756,2021-09-01,Garfield County,3.0,56312.0,45031.0,2258.0,986.0,WA,3.215953,9.72,3.908,0.0,0.0,1.0,1.630454e+09,53023,0
9795,2021-10-01,Garfield County,3.0,55625.0,46026.0,2258.0,986.0,WA,3.215953,9.72,3.908,0.0,0.0,1.0,1.633046e+09,53023,0
9834,2021-11-01,Garfield County,3.0,56437.0,46329.0,2258.0,986.0,WA,3.215953,9.72,3.908,0.0,0.0,1.0,1.635725e+09,53023,0


In [165]:
# Select rows from df2 that do not have a match in df1
df2_unmatched = merged_df[merged_df['state_x'].isna()]
df2_unmatched = df2_unmatched.dropna(axis=1)
col_2 = df2_unmatched.pop('fips')
df2_unmatched = df2_unmatched.assign(fips=col_2)
col_dic = dict(zip(df2_unmatched.columns, column_names))
df2_unmatched = df2_unmatched.rename(columns=col_dic)
df2_unmatched.drop('PHEV', axis=1, inplace=True)

df2_unmatched = df2_unmatched.rename(columns={'Vehicle Count': 'PHEV'})
df2_unmatched = df2_unmatched.assign(BEV=df2_unmatched['PHEV'])
df2_unmatched['PHEV'] = 0

df2_unmatched

,Registration Date,County,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time,fips,BEV
9901,2017-01-01,Grand County,0,54669.0,38747.0,15536.0,6315.0,CO,8.512107,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8049,1.0
9902,2017-01-01,Las Animas County,0,54669.0,38747.0,14323.0,6750.0,CO,3.049501,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8071,1.0
9903,2017-01-01,Montrose County,0,54669.0,38747.0,42280.0,17483.0,CO,19.045188,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8085,1.0
9904,2017-01-01,Park County,0,54669.0,38747.0,18345.0,6987.0,CO,7.927976,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8093,3.0
9905,2017-02-01,Archuleta County,0,54669.0,38747.0,13588.0,5736.0,CO,9.894965,12.66,3.378,0.0,0.0,0.0,1.485907e+09,8007,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11084,2018-10-01,Pend Oreille County,0,54669.0,38747.0,13588.0,5798.0,WA,9.570523,9.53,3.908,0.0,0.0,1.0,1.538352e+09,53051,2.0
11085,2018-10-01,Skamania County,0,54669.0,38747.0,11906.0,4866.0,WA,7.258144,9.53,3.908,0.0,0.0,1.0,1.538352e+09,53059,6.0
11086,2018-11-01,Pacific County,0,54669.0,38747.0,22121.0,9714.0,WA,25.026666,9.53,3.908,0.0,0.0,1.0,1.541030e+09,53049,2.0
11087,2019-01-01,Adams County,0,54669.0,38747.0,19702.0,6035.0,WA,10.708173,9.52,3.908,0.0,0.0,1.0,1.546301e+09,53001,1.0


In [166]:
merged_df = pd.merge(df_phev, df_bev[['Registration Date', 'fips', 'Vehicle Count']], on=['Registration Date','fips'])
merged_df.drop('PHEV', axis=1, inplace=True)
merged_df = merged_df.rename(columns={'Vehicle Count_x': 'PHEV', 'Vehicle Count_y': 'BEV'})
merged_df 

,Registration Date,County,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time,fips,BEV
0,2017-01-01,Adams County,22,54669,38747,509844,167290,CO,445.323695,12.66,3.378,0,0,0,1.483229e+09,8001,24
1,2017-01-01,Arapahoe County,61,54669,38747,649980,241889,CO,821.038725,0.12,2.429,0,0,0,1.483229e+09,8005,65
2,2017-01-01,Boulder County,63,54669,38747,324682,127365,CO,455.351666,12.66,3.378,0,0,0,1.483229e+09,8013,167
3,2017-01-01,Broomfield County,7,54669,38747,69444,27199,CO,2248.011733,0.12,2.429,0,0,0,1.483229e+09,8014,14
4,2017-01-01,Denver County,56,54669,38747,715878,287756,CO,4674.337363,0.12,2.429,0,0,0,1.483229e+09,8031,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8304,2021-12-01,Wahkiakum County,6,63821,47243,4318,1900,WA,16.818489,9.72,3.908,0,0,1,1.638317e+09,53069,15
8305,2021-12-01,Walla Walla County,81,63821,47243,60785,22773,WA,49.277735,9.72,3.908,0,0,1,1.638317e+09,53071,139
8306,2021-12-01,Whatcom County,639,63821,47243,224538,88978,WA,107.617198,9.72,3.908,0,0,1,1.638317e+09,53073,1587
8307,2021-12-01,Whitman County,56,63821,47243,49577,18485,WA,22.217436,9.72,3.908,0,0,1,1.638317e+09,53075,69


In [168]:
df = pd.concat([merged_df, df1_unmatched, df2_unmatched], axis=0)
df.to_csv('./Data/df_all_features_county.csv', index = False)
df

,Registration Date,County,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time,fips,BEV
0,2017-01-01,Adams County,22.0,54669.0,38747.0,509844.0,167290.0,CO,445.323695,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8001,24.0
1,2017-01-01,Arapahoe County,61.0,54669.0,38747.0,649980.0,241889.0,CO,821.038725,0.12,2.429,0.0,0.0,0.0,1.483229e+09,8005,65.0
2,2017-01-01,Boulder County,63.0,54669.0,38747.0,324682.0,127365.0,CO,455.351666,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8013,167.0
3,2017-01-01,Broomfield County,7.0,54669.0,38747.0,69444.0,27199.0,CO,2248.011733,0.12,2.429,0.0,0.0,0.0,1.483229e+09,8014,14.0
4,2017-01-01,Denver County,56.0,54669.0,38747.0,715878.0,287756.0,CO,4674.337363,0.12,2.429,0.0,0.0,0.0,1.483229e+09,8031,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11084,2018-10-01,Pend Oreille County,0.0,54669.0,38747.0,13588.0,5798.0,WA,9.570523,9.53,3.908,0.0,0.0,1.0,1.538352e+09,53051,2.0
11085,2018-10-01,Skamania County,0.0,54669.0,38747.0,11906.0,4866.0,WA,7.258144,9.53,3.908,0.0,0.0,1.0,1.538352e+09,53059,6.0
11086,2018-11-01,Pacific County,0.0,54669.0,38747.0,22121.0,9714.0,WA,25.026666,9.53,3.908,0.0,0.0,1.0,1.541030e+09,53049,2.0
11087,2019-01-01,Adams County,0.0,54669.0,38747.0,19702.0,6035.0,WA,10.708173,9.52,3.908,0.0,0.0,1.0,1.546301e+09,53001,1.0


# Extract 1 Month for GNN

In [41]:
df = pd.read_csv('./Data/df_all_features_county.csv')
# Desired prediction var
predict_label = ['PHEV', 'BEV']

# Drop zip code since it would increase the number of features by ~4k
# also zip code is highly correlated to population and income
# Drop registration date since we will change to Unix timestamps
drop_col = ['state','County', 'Registration Date', 'fips', 'Unix Time']

# Get labels of all features
features = [c for c in df.columns if c not in predict_label + drop_col]
features

['Average EV Price',
 'New Car Average',
 'population',
 'household_income',
 'POPDEN_COU',
 'Electricity Price',
 'Gas Price',
 'NY',
 'TX',
 'WA']

In [42]:
df

,Registration Date,County,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time,fips,BEV
0,2017-01-01,Adams County,22.0,54669.0,38747.0,509844.0,167290.0,CO,445.323695,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8001,24.0
1,2017-01-01,Arapahoe County,61.0,54669.0,38747.0,649980.0,241889.0,CO,821.038725,0.12,2.429,0.0,0.0,0.0,1.483229e+09,8005,65.0
2,2017-01-01,Boulder County,63.0,54669.0,38747.0,324682.0,127365.0,CO,455.351666,12.66,3.378,0.0,0.0,0.0,1.483229e+09,8013,167.0
3,2017-01-01,Broomfield County,7.0,54669.0,38747.0,69444.0,27199.0,CO,2248.011733,0.12,2.429,0.0,0.0,0.0,1.483229e+09,8014,14.0
4,2017-01-01,Denver County,56.0,54669.0,38747.0,715878.0,287756.0,CO,4674.337363,0.12,2.429,0.0,0.0,0.0,1.483229e+09,8031,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11084,2018-10-01,Pend Oreille County,0.0,54669.0,38747.0,13588.0,5798.0,WA,9.570523,9.53,3.908,0.0,0.0,1.0,1.538352e+09,53051,2.0
11085,2018-10-01,Skamania County,0.0,54669.0,38747.0,11906.0,4866.0,WA,7.258144,9.53,3.908,0.0,0.0,1.0,1.538352e+09,53059,6.0
11086,2018-11-01,Pacific County,0.0,54669.0,38747.0,22121.0,9714.0,WA,25.026666,9.53,3.908,0.0,0.0,1.0,1.541030e+09,53049,2.0
11087,2019-01-01,Adams County,0.0,54669.0,38747.0,19702.0,6035.0,WA,10.708173,9.52,3.908,0.0,0.0,1.0,1.546301e+09,53001,1.0


In [43]:
month = '2021-04-01'
df_m = df[df['Registration Date'] == month]
df_m = df_m.reset_index(drop=True)
df_m

,Registration Date,County,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time,fips,BEV
0,2021-04-01,Adams County,677.0,56059.0,41172.0,509844.0,167290.0,CO,445.323695,13.450,3.378,0.0,0.0,0.0,1.617235e+09,8001,1090.0
1,2021-04-01,Arapahoe County,1359.0,56059.0,41172.0,649980.0,241889.0,CO,821.038725,0.141,2.908,0.0,0.0,0.0,1.617235e+09,8005,3250.0
2,2021-04-01,Archuleta County,53.0,56059.0,41172.0,13588.0,5736.0,CO,9.894965,13.450,3.378,0.0,0.0,0.0,1.617235e+09,8007,9.0
3,2021-04-01,Boulder County,1391.0,56059.0,41172.0,324682.0,127365.0,CO,455.351666,13.450,3.378,0.0,0.0,0.0,1.617235e+09,8013,4393.0
4,2021-04-01,Broomfield County,201.0,56059.0,41172.0,69444.0,27199.0,CO,2248.011733,0.141,2.908,0.0,0.0,0.0,1.617235e+09,8014,676.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,2021-04-01,Somervell County,0.0,56059.0,41172.0,8958.0,3145.0,TX,49.389389,10.770,2.551,0.0,1.0,0.0,1.617235e+09,48425,30.0
293,2021-04-01,Sutton County,0.0,56059.0,41172.0,3793.0,1282.0,TX,2.319236,10.770,2.551,0.0,1.0,0.0,1.617235e+09,48435,17.0
294,2021-04-01,Upshur County,0.0,56059.0,41172.0,41386.0,14392.0,TX,70.143470,10.770,2.551,0.0,1.0,0.0,1.617235e+09,48459,14.0
295,2021-04-01,Val Verde County,0.0,56059.0,41172.0,49018.0,16388.0,TX,15.131973,10.770,2.551,0.0,1.0,0.0,1.617235e+09,48465,37.0


In [49]:
# extract only the adjacency in our month's graph
fips_list = df_m['fips'].to_list()

# map dict
idx_list = list(range(len(df_m)))

mapping_dic = dict(zip(fips_list, idx_list))
mapping_dic

{8001: 0,
 8005: 1,
 8007: 2,
 8013: 3,
 8014: 4,
 8015: 5,
 8029: 6,
 8031: 7,
 8035: 8,
 8037: 9,
 8041: 10,
 8039: 11,
 8043: 12,
 8045: 13,
 8049: 14,
 8051: 15,
 8059: 16,
 8067: 17,
 8069: 18,
 8075: 19,
 8077: 20,
 8083: 21,
 8085: 22,
 8087: 23,
 8093: 24,
 8097: 25,
 8101: 26,
 8105: 27,
 8107: 28,
 8113: 29,
 8117: 30,
 8119: 31,
 8123: 32,
 36001: 33,
 36003: 34,
 36005: 35,
 36007: 36,
 36011: 37,
 36013: 38,
 36015: 39,
 36017: 40,
 36019: 41,
 36021: 42,
 36023: 43,
 36025: 44,
 36027: 45,
 36029: 46,
 36031: 47,
 36033: 48,
 36035: 49,
 36037: 50,
 36039: 51,
 36043: 52,
 36045: 53,
 36047: 54,
 36051: 55,
 36053: 56,
 36055: 57,
 36057: 58,
 36059: 59,
 36061: 60,
 36063: 61,
 36065: 62,
 36067: 63,
 36069: 64,
 36071: 65,
 36073: 66,
 36075: 67,
 36077: 68,
 36079: 69,
 36081: 70,
 36083: 71,
 36085: 72,
 36087: 73,
 36091: 74,
 36093: 75,
 36097: 76,
 36089: 77,
 36101: 78,
 36103: 79,
 36105: 80,
 36107: 81,
 36109: 82,
 36111: 83,
 36113: 84,
 36115: 85,
 36117: 86,

In [50]:
# store features for each node, n_node x n_features
df_X_gnn = df_m[features]
df_X_gnn.to_csv('./Data/df_X_gnn.csv', index = False)
df_X_gnn

,Average EV Price,New Car Average,population,household_income,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA
0,56059.0,41172.0,509844.0,167290.0,445.323695,13.450,3.378,0.0,0.0,0.0
1,56059.0,41172.0,649980.0,241889.0,821.038725,0.141,2.908,0.0,0.0,0.0
2,56059.0,41172.0,13588.0,5736.0,9.894965,13.450,3.378,0.0,0.0,0.0
3,56059.0,41172.0,324682.0,127365.0,455.351666,13.450,3.378,0.0,0.0,0.0
4,56059.0,41172.0,69444.0,27199.0,2248.011733,0.141,2.908,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
292,56059.0,41172.0,8958.0,3145.0,49.389389,10.770,2.551,0.0,1.0,0.0
293,56059.0,41172.0,3793.0,1282.0,2.319236,10.770,2.551,0.0,1.0,0.0
294,56059.0,41172.0,41386.0,14392.0,70.143470,10.770,2.551,0.0,1.0,0.0
295,56059.0,41172.0,49018.0,16388.0,15.131973,10.770,2.551,0.0,1.0,0.0


In [51]:
# output values to be predicted at each node, n_node x n_pred
df_y_gnn = df_m[['PHEV', 'BEV']]
df_y_gnn.to_csv('./Data/df_y_gnn.csv', index = False)
df_y_gnn

,PHEV,BEV
0,677.0,1090.0
1,1359.0,3250.0
2,53.0,9.0
3,1391.0,4393.0
4,201.0,676.0
...,...,...
292,0.0,30.0
293,0.0,17.0
294,0.0,14.0
295,0.0,37.0


In [52]:
# read adjacency
df_a = pd.read_csv('./Data/df_adjacency.csv')
df_a

,countyname,fipscounty,neighborname,fipsneighbor,county,state,neighbor_county,neighbor_state,count,distance_(mi)
0,"Adams County, CO",8001,"Adams County, CO",8001,Adams County,CO,Adams County,CO,1,0.000000
1,"Adams County, CO",8001,"Arapahoe County, CO",8005,Adams County,CO,Arapahoe County,CO,1,16.600485
2,"Adams County, CO",8001,"Broomfield County, CO",8014,Adams County,CO,Broomfield County,CO,1,683.581802
3,"Adams County, CO",8001,"Denver County, CO",8031,Adams County,CO,Denver County,CO,1,649.967579
4,"Adams County, CO",8001,"Jefferson County, CO",8059,Adams County,CO,Jefferson County,CO,1,56.499723
...,...,...,...,...,...,...,...,...,...,...
2885,"Yakima County, WA",53077,"Klickitat County, WA",53039,Yakima County,WA,Klickitat County,WA,1,37.217994
2886,"Yakima County, WA",53077,"Lewis County, WA",53041,Yakima County,WA,Lewis County,WA,1,79.821283
2887,"Yakima County, WA",53077,"Pierce County, WA",53053,Yakima County,WA,Pierce County,WA,1,81.294253
2888,"Yakima County, WA",53077,"Skamania County, WA",53059,Yakima County,WA,Skamania County,WA,1,65.230716


In [53]:
# only take out adjacency attributes if the 
df_e = df_a[df_a['fipscounty'].isin(fips_list)]
df_e = df_e[df_e['fipsneighbor'].isin(fips_list)]

# drop rows where the fipscounty = fipsneighbor, i.e. distance to itself is 0
df_e = df_e[df_e['distance_(mi)'] != 0]
df_e

,countyname,fipscounty,neighborname,fipsneighbor,county,state,neighbor_county,neighbor_state,count,distance_(mi)
1,"Adams County, CO",8001,"Arapahoe County, CO",8005,Adams County,CO,Arapahoe County,CO,1,16.600485
2,"Adams County, CO",8001,"Broomfield County, CO",8014,Adams County,CO,Broomfield County,CO,1,683.581802
3,"Adams County, CO",8001,"Denver County, CO",8031,Adams County,CO,Denver County,CO,1,649.967579
4,"Adams County, CO",8001,"Jefferson County, CO",8059,Adams County,CO,Jefferson County,CO,1,56.499723
5,"Adams County, CO",8001,"Morgan County, CO",8087,Adams County,CO,Morgan County,CO,1,34.820428
...,...,...,...,...,...,...,...,...,...,...
2884,"Yakima County, WA",53077,"Kittitas County, WA",53037,Yakima County,WA,Kittitas County,WA,1,48.305737
2885,"Yakima County, WA",53077,"Klickitat County, WA",53039,Yakima County,WA,Klickitat County,WA,1,37.217994
2886,"Yakima County, WA",53077,"Lewis County, WA",53041,Yakima County,WA,Lewis County,WA,1,79.821283
2887,"Yakima County, WA",53077,"Pierce County, WA",53053,Yakima County,WA,Pierce County,WA,1,81.294253


In [54]:
# list of edge indices, 2 x n_edges
df_e_idx = df_e[['fipscounty', 'fipsneighbor']]
df_e_idx = df_e_idx.replace(mapping_dic)
df_e_idx = df_e_idx.T
df_e_idx.to_csv('./Data/df_e_idx_gnn.csv', index = False)
df_e_idx

,1,2,3,4,5,7,14,16,17,18,...,2878,2879,2881,2882,2883,2884,2885,2886,2887,2888
fipscounty,0,0,0,0,0,0,1,1,1,1,...,217,217,218,218,218,218,218,218,218,218
fipsneighbor,1,4,7,16,23,32,0,7,8,11,...,201,211,183,192,196,198,199,200,206,209


In [57]:
df_e_idx = df_e[['fipscounty', 'fipsneighbor']]
df_e_idx

,fipscounty,fipsneighbor
1,8001,8005
2,8001,8014
3,8001,8031
4,8001,8059
5,8001,8087
...,...,...
2884,53077,53037
2885,53077,53039
2886,53077,53041
2887,53077,53053


In [55]:
# list of edge attributes, n_edges x n_edge_attributes
df_e_att = df_e['distance_(mi)']
df_e_att.to_csv('./Data/df_e_att_gnn.csv', index = False)
df_e_att

1        16.600485
2       683.581802
3       649.967579
4        56.499723
5        34.820428
           ...    
2884     48.305737
2885     37.217994
2886     79.821283
2887     81.294253
2888     65.230716
Name: distance_(mi), Length: 1250, dtype: float64

# Adjacency

In [57]:
# state_order = ['CO', 'NY', 'TX', 'WA']
# # drop counties that have nan in state
# df_c.dropna(subset=['state'], inplace=True)
# df_c
# df_c['state'].value_counts()['TX']
# df_a = pd.read_csv('./Data/county_adjacency2010.csv')
# df_a
# df_a[['county','state']] = df_a['countyname'].str.split(', ', expand=True)
# df_a[['neighbor_county', 'neighbor_state']] = df_a['neighborname'].str.split(', ', expand=True)
# df_a = df_a[df_a['state'].isin(state_order)]
# df_a
# # Create a geolocator object
# geolocator = Nominatim(user_agent='my_app')

# # Define a function to calculate distance between two counties
# def calculate_distance(row):
#     # Look up latitude and longitude coordinates for county and neighbor
#     county = geolocator.geocode(row['county'] + ', ' + row['state'] + ', USA')
#     neighbor = geolocator.geocode(row['neighbor_county'] + ', ' + row['neighbor_state'] + ', USA')
#     # Calculate distance using Haversine formula
#     try:
#         distance = geodesic((county.latitude, county.longitude), (neighbor.latitude, neighbor.longitude)).miles
#     except:
#         distance = 0
#     return distance

# # Apply the function to each row of the dataframe
# df_a['distance_(mi)'] = df_a.apply(calculate_distance, axis=1)
# df_a.to_csv('./Data/df_adjacency.csv', index = False)
# df_a
# df_zeros = df_a[df_a['distance_(mi)'] == 0]
# df_zeros.to_csv('./Data/df_zeros.csv', index = False)
# nm_rows = df_a[df_a['neighborname'].str.contains('NM', na=False)]
# len(nm_rows)
# # Create a dataframe that lists all neighboring county pairs
# df_a['count'] = 1
# neighbor_pairs = df_a[['fipscounty', 'fipsneighbor', 'count', 'distance_(mi)']]

# # Create a pivot table with neighboring county names as row and column indices
# adjacency_matrix = df_a.pivot_table(index='fipscounty', columns='fipsneighbor', values='count', fill_value=0)

# # Convert the pivot table to a numpy array to get the adjacency matrix
# distance_matrix = df_a.pivot_table(index='fipscounty', columns='fipsneighbor', values='distance_(mi)', fill_value=0)

,Registration Date,County,Vehicle Count,Average EV Price,New Car Average,population,household_income,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,PHEV,Unix Time
0,2017-01-01,Adams County,24,54669,38747,509844,167290,445.323695,12.66,3.378,0,0,0,0,1.483229e+09
1,2017-01-01,Arapahoe County,66,54669,38747,649980,241889,821.038725,0.12,2.429,0,0,0,0,1.483229e+09
2,2017-01-01,Boulder County,170,54669,38747,324682,127365,455.351666,12.66,3.378,0,0,0,0,1.483229e+09
3,2017-01-01,Broomfield County,15,54669,38747,69444,27199,2248.011733,0.12,2.429,0,0,0,0,1.483229e+09
4,2017-01-01,Denver County,95,54669,38747,715878,287756,4674.337363,0.12,2.429,0,0,0,0,1.483229e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17001,2019-01-01,Thurston County,84,54669,38747,284698,112323,408.022371,9.52,3.908,0,0,1,1,1.546301e+09
17002,2019-01-01,Walla Walla County,6,54669,38747,60785,22773,49.277735,9.52,3.908,0,0,1,1,1.546301e+09
17003,2019-01-01,Whatcom County,35,54669,38747,224538,88978,107.617198,9.52,3.908,0,0,1,1,1.546301e+09
17004,2019-01-01,Whitman County,5,54669,38747,49577,18485,22.217436,9.52,3.908,0,0,1,1,1.546301e+09
